# Milestone 2
This notebook contains the classifier code for MS 2. For more info, see for example: [the sklearn docs](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html#training-a-classifier)

In [1]:
from utils import get_data
from sklearn import metrics
from sklearn.pipeline import Pipeline

X_train, y_train, X_test, y_test = get_data()

loaded df from ../../data/dreaddit-train.csv with 115 columns.
number of na values in the whole df: 0
loaded df from ../../data/dreaddit-test.csv with 115 columns.
number of na values in the whole df: 0


In [2]:
from sklearn.base import BaseEstimator
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV

def train_and_eval(clf: BaseEstimator,
                   gs_params: dict,
                   X_train: list[str],
                   y_train: list[int],
                   X_test: list[str],
                   y_test: list[int],) -> str:
    text_clf = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', clf),
    ])

    if gs_params:
        # do gridsearch if params are provided
        text_clf = GridSearchCV(text_clf, gs_params, cv=5, n_jobs=-1)

    text_clf.fit(X_train, y_train)
    y_pred = text_clf.predict(X_test)

    for param_name in sorted(gs_params.keys()):
        print("%s: %r" % (param_name, text_clf.best_params_[param_name]))

    return metrics.classification_report(y_test, y_pred, target_names=['no stress', 'stress'])

## Naive Bayes

In [3]:
from sklearn.naive_bayes import MultinomialNB

res = train_and_eval(MultinomialNB(),
                     {},
                     X_train,
                     y_train,
                     X_test,
                     y_test)
print(res)

              precision    recall  f1-score   support

   no stress       0.90      0.30      0.45       346
      stress       0.60      0.97      0.74       369

    accuracy                           0.65       715
   macro avg       0.75      0.64      0.60       715
weighted avg       0.74      0.65      0.60       715



## RidgeClassifier

In [4]:
from sklearn.linear_model import RidgeClassifier

res = train_and_eval(RidgeClassifier(tol=1e-2, solver="sparse_cg"),
                     {},
                     X_train,
                     y_train,
                     X_test,
                     y_test)
print(res)

              precision    recall  f1-score   support

   no stress       0.70      0.68      0.69       346
      stress       0.71      0.72      0.72       369

    accuracy                           0.70       715
   macro avg       0.70      0.70      0.70       715
weighted avg       0.70      0.70      0.70       715



## SVM

In [5]:
from sklearn.linear_model import SGDClassifier

res = train_and_eval(SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None),
                     {},
                     X_train,
                     y_train,
                     X_test,
                     y_test)
print(res)

              precision    recall  f1-score   support

   no stress       0.73      0.66      0.69       346
      stress       0.71      0.77      0.74       369

    accuracy                           0.72       715
   macro avg       0.72      0.71      0.71       715
weighted avg       0.72      0.72      0.72       715



## SVM __with__ Grid Search tuning

In [6]:
from sklearn.linear_model import SGDClassifier

parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'tfidf__use_idf': (True, False),
    'clf__loss': ('hinge', 'perceptron'),
    'clf__alpha': (1e-1, 1e-2, 1e-3, 1e-4),
}

res = train_and_eval(SGDClassifier(),
                     parameters,
                     X_train,
                     y_train,
                     X_test,
                     y_test)
print(res)

clf__alpha: 0.001
clf__loss: 'hinge'
tfidf__use_idf: True
vect__ngram_range: (1, 1)
              precision    recall  f1-score   support

   no stress       0.74      0.66      0.70       346
      stress       0.71      0.78      0.74       369

    accuracy                           0.72       715
   macro avg       0.72      0.72      0.72       715
weighted avg       0.72      0.72      0.72       715

